This is a Project for the EE482 course (Engineering applications in AI) in KAU

The project is basically our participation in the Titanic challenge on Kaggle. The goal of the challenge is to predict weither the passenger survived or not

In [24]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

The Next segment is for loading and cleaning the data. Some features are irrelevent which will be dropped and others aren't fully complete so we need to fill them. Also, some of the data are in string so we are going to use sklearn to help us preproccess the data. We should also change some data from columns to numbers that the computer can work with. one way is by using .replace or .map and the other by the help of sklearn preprocessing feature.

In [128]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
passids = test["PassengerId"]

def cleanData(data):
    

    # Remove unwanted features
    data = data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', "Embarked"], axis=1) 


    # Fill incompleted data
    columns = ['SibSp', 'Parch', 'Fare', "Age"]
    for col in columns:
        data[col].fillna(data[col].median(), inplace=True)
    return data


train = cleanData(train)
test = cleanData(test)

#Encode string data
train['Sex'] = train['Sex'].map({'male': 1, 'female': 0})
test['Sex'] = test['Sex'].map({'male': 1, 'female': 0})


print(train.head())

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare
0         0       3    1  22.0      1      0   7.2500
1         1       1    0  38.0      1      0  71.2833
2         1       3    0  26.0      0      0   7.9250
3         1       1    0  35.0      1      0  53.1000
4         0       3    1  35.0      0      0   8.0500


Now that we have the data ready for creating the model. All we have to do is split it into training and validation. 

In [198]:
y= train['Survived']
X = train.drop('Survived', axis = 1) # Now that we are going to split and getting ready for training we should seperate the labels and features


X_t, X_v, y_t, y_v = train_test_split(X, y, test_size=0.2, random_state=42)

We chose to create a multilayer neural network which is suitable for binary classification problems. After trail and error we found that these are the best hyperparameters we found.


6 hidden layers with 5 perceptrons each  ||  Activation function -> Relu  ||  Weight optimization -> adam  ||  Learning rate -> 0.01



In [199]:
mlp = MLPClassifier(hidden_layer_sizes=(5, 6), learning_rate_init=0.01, random_state=42).fit(X_t,y_t)

Accuracy = mlp.score(X_v, y_v)
print(f"Validation accuracy: {Accuracy * 100:.5f}%")

Validation accuracy: 82.12291%


In [200]:
Submission = mlp.predict(test)

preds = pd.DataFrame({"PassengerId":passids.values,
                    "Survived": Submission,})

preds.to_csv("Submission.csv",index = False)